# Guide to using Kramers-Kronig method

Initial importing of libraries and modules:

In [ ]:
# Preliminaries
from spectrum import *
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pandas as pd
import numpy as np
current_path = Path.cwd()

Loading reference data set into a pandas dataframe using ``data_macro``:

In [ ]:
pathMagneto = os.path.join(current_path, "Data", "Facet_001")
inst_MO = Data_macro(auto = True, data_path = pathMagneto, units = "cm-1", data_head = "none", ref_head = "none", zero_field = True, flag_txt = False)
df_macro = inst_MO.Auto()

#generating more sparse dataframe for improved processing time
df = df_macro.loc[::15,5.0]

To carry out the Kramers-Kronig relation calculations, we use the method:<br> ``Treament.Kramers_Kronig()``<br>
``Kramers_Kronig(df, model, low_lim, high_lim, below_fe, b)``
<br>
- ``df``: (pandas dataframe) The input dataframe of spectral data to be processed.<br><br>
- ``model``: (string) The model to be used to for low energy extrapolation.<br>("Hagen-Rubens", "Insulator", "Power law", "Metal", "Marginal Fermi liquid", "Gorter-Casimir two-fluid model", or "Superconducting")<br><br>
- ``low_lim``: (float) Limit of low energy extrapolation.<br><br>
- ``high_lim``: (float) Limit of high energy extrapolation.<br><br>
- ``below_fe``: (Boolean) Specifies whether the energy range is below the free-electron (plasma) frequency.<br><br>
- ``b``: (float) Additional parameter for models that require an additional degree of freedom - "Insulator" and "Power law". 

In [ ]:
proc_df = Treatment.Kramers_Kronig(df, model = "Hagen-Rubens", low_lim = 405, high_lim = 6995, below_fe = False)
proc_df

# Kramers-Kronig widget
<br>The following code assumes that input cropped dataframe is named ``df`` and has the index in units of Hz.


In [ ]:
# Preliminaries
from IPython.display import display, clear_output
import ipywidgets as widgets

class InteractiveUI:
    def __init__(self, df):
        self.df = df
        e_max = self.df.index.max()

        self.options = ["Hagen-Rubens", "Insulator", "Power law", "Metal", "Marginal Fermi Liquid", "Gorter-Casimir two-fluid model", "Superconducting"]
        self.dropdown = widgets.Dropdown(options=self.options, description="Select model:")

        self.slider1 = widgets.FloatSlider(min=0.0, max=e_max, value=500.0, description="Low energy extrapolation limit:")
        self.slider1_val = widgets.Label(value=str(self.slider1.value))
        self.slider1_input = widgets.FloatText(value=self.slider1.value)
        
        self.slider2 = widgets.FloatSlider(min=0.0, max=e_max, value=6000.0, description="High energy extrapolation limit:")
        self.slider2_val = widgets.Label(value=str(self.slider2.value))
        self.slider2_input = widgets.FloatText(value=self.slider2.value)

        self.text_input = widgets.Text()
        self.text_input.layout.display = 'none'

        self.checkbox = widgets.Checkbox(value=False, description='Measurement is below plasma frequency:')

        self.button = widgets.Button(description="Run KK")
        self.button.on_click(self.button_click)

        self.dropdown.observe(self.dropdown_event_handler, names='value')
        self.slider1.observe(self.slider1_event_handler, names='value')
        self.slider1_input.observe(self.slider1_input_event_handler, names='value')
        self.slider2.observe(self.slider2_event_handler, names='value')
        self.slider2_input.observe(self.slider2_input_event_handler, names='value')

        self.update_ui()

    def dropdown_event_handler(self, change):
        self.update_ui()

    def slider1_event_handler(self, change):
        self.slider1_val.value = str(change.new)
        self.slider1_input.value = change.new

    def slider1_input_event_handler(self, change):
        try:
            new_value = float(change.new)
            if self.slider1.min <= new_value <= self.slider1.max:
                self.slider1.value = new_value
                self.slider1_val.value = str(new_value)
        except ValueError:
            pass

    def slider2_event_handler(self, change):
        self.slider2_val.value = str(change.new)
        self.slider2_input.value = change.new

    def slider2_input_event_handler(self, change):
        try:
            new_value = float(change.new)
            if self.slider2.min <= new_value <= self.slider2.max:
                self.slider2.value = new_value
                self.slider2_val.value = str(new_value)
        except ValueError:
            pass

    def hide_widgets(self, *widgets):
        for widget in widgets:
            widget.layout.display = 'none'

    def display_widgets(self, *widgets):
        for widget in widgets:
            widget.layout.display = 'block'

    def button_click(self, button):
        model = self.dropdown.value
        e_low = self.slider1.value
        e_high = self.slider2.value
    
        if model in ["Insulator", "Power law"]:
            try:
                float(self.text_input.value)
            except ValueError:
                print ("Error: value entered must be a float")
            b = float(self.text_input.value)
        else:
            b = None

        below_fe = self.checkbox.value
        if e_low <= e_high:
            proc_df = Treatment.Kramers_Kronig(df, model=model, low_lim=e_low, high_lim=e_high, below_fe=below_fe, b=b)
            print(proc_df)
        else:
            print("Error: energy range entered is invalid")

    def update_ui(self):
        clear_output(wait=True)
        
        selected_option = self.dropdown.value
        if selected_option in ["Insulator", "Power law"]:
            self.display_widgets(self.slider1, self.slider1_val, self.slider1_input, self.slider2, self.slider2_val, self.slider2_input, self.text_input, self.checkbox, self.button)
            if selected_option == "Insulator":
                self.text_input.description = "Constant:"
            elif selected_option == "Power law":
                self.text_input.description = "Exponent:"
            self.text_input.layout.visibility = 'visible'
        else:
            self.display_widgets(self.slider1, self.slider1_val, self.slider1_input, self.slider2, self.slider2_val, self.slider2_input, self.checkbox, self.button)
            self.hide_widgets(self.text_input)

        display(self.dropdown, self.slider1, self.slider1_val, self.slider1_input, self.slider2, self.slider2_val, self.slider2_input, self.text_input, self.checkbox, self.button)

# Create an instance of the InteractiveUI class
ui = InteractiveUI(df)
